** 5-1. 딥러닝 개요 **
** 5-2. TensorFlow 설치하기 **
** 5-3. Jupyter Notebook **
** 5-4. TensorFlow 기본 **

### 5-1 딥러닝 개요
* 딥러닝은 '특징량 추출'을 자동으로.
* 2012년 ILSVRC (ImageNet ...) : 100만장의 이미지/100개의 물체 : AlexNet (87%)
* 인공신경망(Artificial Neural Network, ANN) - 1940년대 (1943년)
    + 인간의 뇌가 패턴을 인식하는 방식을 모사한 알고리즘 (인간의 신경망 모사)
* 퍼셉트론(Perceptron) - 1958년
    + 선형분류를 수행할 수 있는 피드포워드 뉴럴네트워크
    + ![Image](./Perceptron.png)
    + ![Image](./xor.png)
* Multi-Layer Perceptrons(MLP), Backpropagation Algorithm - 1980년대
* 딥러닝 - 현재 : 빅데이터, CPU성능, GPU, 전처리, dropout
    + ![Image](./ann.png)
* DNN(Deep Neural Network) : 3개 이상 중첩된 신경망
* 학습이란? 뉴런의 접속 가중치를 조정하는것.



### 5-2 TensorFlow
- 대규모 숫자 계산을 해주는 C++로 만들어진 라이브러리
- 최대 장점은 '자료', '커뮤니티'가 많다는 것

## docker에 anaconda 설치
> docker pull continuumio/anaconda3 (13분)
![Image](./docker-pull.png)

> docker run -i -t continuumio/anaconda3 /bin/bash

> docker run -i -t -p 8866:8866 continuumio/anaconda3 /bin/bash -c "/opt/conda/bin/conda install jupyter -y --quiet && mkdir /opt/notebooks && /opt/conda/bin/jupyter notebook --notebook-dir=/opt/notebooks --ip='*' --port=8866 --no-browser --allow-root"



### 5-3 Jupyter Notebook
- 설치 (anaconda에 포함)
> pip install jupyter
- 실행
> jupyter notebook
- 새노트 만들기
> 왼쪽 상단 new버튼 클릭
- Cell 추가하기
> Insert -> Insert Cell Below (B)
- Cell 실행하기
> Cell -> Run Cells (ctrl + enter)
- 주석과 문서 입력하기
> Cell -> Cell Type -> Markdown (M)
> markdown 문법 사용 [https://gist.github.com/ihoneymon/652be052a0727ad59601]
- 노트 저장
> File -> Save and Checkpoint
- 노트 복구
> File -> Revert to Checkpoint
- 시각화
> %matplotlib inline
- tensorflow 설체
> !pip install tensorflow



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
x = np.arange(-20, 20, 0.1)
y = np.sin(x)
plt.plot(x,y)

### 5-4 TensorFlow 기본
- 데이터 플로우 그래프 (연산 정의) -> 세션 실행.

In [ ]:
# 상수식 연산
import tensorflow as tf

a = tf.constant(120, name='a')
b = tf.constant(130, name='b')
c = tf.constant(140, name='c')

v = tf.Variable(0, name='v')

calc_op = a + b + c
print(calc_op)

assign_op = tf.assign(v, calc_op)

with tf.Session() as sess:
    sess.run(assign_op)
    print(sess.run(v))

In [ ]:
# placeholder 사용
a = tf.placeholder(tf.int32, [3])
b = tf.constant(2)
x2_op = a * b

with tf.Session() as sess:
    r1 = sess.run(x2_op, feed_dict={a:[1,2,3]})
    print(r1)
    r2 = sess.run(x2_op, feed_dict={a:[10,20,30]})
    print(r2)

In [ ]:
# placeholder None사용
a = tf.placeholder(tf.int32, [None])
b = tf.constant(2)
x2_op = a * b

with tf.Session() as sess:
    r1 = sess.run(x2_op, feed_dict={a:[1,2,3]})
    print(r1)
    r2 = sess.run(x2_op, feed_dict={a:[10,20,30,40,50]})
    print(r2)

## BMI 처리
![Image](./bmi.png)

In [ ]:
# bmi 처리
import pandas as pd
import numpy as np
import tensorflow as tf

csv = pd.read_csv('./bmi.csv')

# 정규화
csv['height'] = csv['height'] / 200
csv['weight'] = csv['weight'] / 100

# one hot encoding
bclass = {'thin':[1,0,0], 'normal':[0,1,0], 'fat':[0,0,1]}
csv['label_pat'] = csv['label'].apply(lambda x : np.array(bclass[x]))

# a = csv[:5]
# print(a)
# print(a[['weight', 'height']])
# print(list(a[['weight', 'height']]))
# print(a['label_pat'])
# print(list(a['label_pat']))
# return

test_csv = csv[15000:20000]
test_pat = test_csv[['weight', 'height']]
test_ans = list(test_csv['label_pat'])

x  = tf.placeholder(tf.float32, [None, 2])
y_ = tf.placeholder(tf.float32, [None, 3])

W = tf.Variable(tf.zeros([2,3]))
b = tf.Variable(tf.zeros([3]))
y = tf.nn.softmax(tf.matmul(x,W) + b)

cost = -tf.reduce_sum(y_ * tf.log(y))
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(cost)

predict = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(3500):
        i = (step * 100) % 1000
        rows = csv[i + 1: i + 1 + 100]
        x_pat = rows[['weight', 'height']]
        y_ans = list(rows['label_pat'])
        fd = {x:x_pat, y_:y_ans}
        sess.run(train, feed_dict=fd)
        if step % 500 == 0:
            cre = sess.run(cost, feed_dict=fd)
            acc = sess.run(accuracy, feed_dict={x:test_pat, y_:test_ans})
            print('step=', step, 'cre=', cre, 'acc=', acc)
    
#     r = sess.run(y, feed_dict={x:test_pat})
#     print(r)
#     print(test_ans[:3])
#     print(test_ans[-3:])
    acc = sess.run(accuracy, feed_dict={x:test_pat, y_:test_ans}) 
    print('정답률:', acc)

![Image](./softmax.png)
![Image](./cross_entropy.png)